In [3]:
import pandas as pd
from sqlalchemy import create_engine
from secrets import settings

In [7]:
engine = create_engine(settings['skills_db'])
# JMLR
df = pd.read_sql_query('select title, skills, data_skills from "ContentJMLR"', engine)
df['content'] = 'JMLR'
# Youtube
df_t = pd.read_sql_query('select title, skills, data_skills from "ContentYoutube"', engine)
df_t['content'] = 'Youtube'
df = df.append(df_t)
# Medium
df_t = pd.read_sql_query('select title, skills, data_skills from "ContentMedium"', engine)
df_t['content'] = 'Medium'
df = df.append(df_t)
# KDnuggets
df_t = pd.read_sql_query('select title, skills, data_skills from "ContentKDnuggets"', engine)
df_t['content'] = 'KDnuggets'
df = df.append(df_t)
# FreeCodeCamp
df_t = pd.read_sql_query('select title, skills, data_skills from "ContentFreeCodeCampCourse"', engine)
df_t['content'] = 'FreeCodeCamp'
df = df.append(df_t)
engine.dispose()
df = df.reset_index(drop=True)
df.head()

,title,skills,data_skills,content
0,On the Optimality of Kernel-Embedding Based Go...,TestNG; Testing,None,JMLR
1,Domain Generalization by Marginal Transfer Lea...,Algorithm; Analysis; Supervised Learning; Trai...,None,JMLR
2,Regulating Greed Over Time in Multi-Armed Bandits,Algorithm; Analysis; Exploit; Retail; Sentry; ...,None,JMLR
3,An Empirical Study of Bayesian Optimization: A...,Accounting; Algorithm; Bayesian Optimization; ...,None,JMLR
4,The Decoupled Extended Kalman Filter for Dynam...,Exploit; Modelling; Uncertainty,None,JMLR


In [8]:
df['content'].value_counts()

Medium          12356
Youtube          5565
KDnuggets         831
JMLR              290
FreeCodeCamp       46
Name: content, dtype: int64

In [9]:
skill_list = []

for _, row in df.iterrows():
    data_skills = row['data_skills']
    if not isinstance(data_skills, str):
        continue
    data_skills = data_skills.split('; ')
    for s in data_skills:
        skill_list.append({'skill': s, 'content': row['content']})

df_s = pd.DataFrame.from_dict(skill_list)
df_s.head()

,skill,content
0,AI,JMLR
1,R programming,JMLR
2,Unsupervised Machine Learning,JMLR
3,Visualizations,JMLR
4,Regressions,JMLR


In [10]:
df_s2 = df_s.value_counts().reset_index()
df_s2 = df_s2.pivot(index='skill', columns='content', values=0)
df_s2 = df_s2.fillna(0)
df_s2['Total'] = df_s2.sum(axis=1)
df_s2.to_csv('DataSkillContentCount.csv')
df_s2

content,FreeCodeCamp,JMLR,KDnuggets,Medium,Youtube,Total
skill,,,,,,
AI,2.0,33.0,206.0,1812.0,473.0,2526.0
Apache,0.0,0.0,0.0,19.0,48.0,67.0
C++,0.0,3.0,1.0,6.0,162.0,172.0
Cloud Technologies,0.0,0.0,10.0,135.0,75.0,220.0
Computer Vision,0.0,0.0,21.0,70.0,150.0,241.0
Dashboards,0.0,0.0,4.0,52.0,192.0,248.0
Data Cleansing / Preparation,0.0,0.0,28.0,20.0,125.0,173.0
Database Management System (DBMS),0.0,0.0,0.0,2.0,161.0,163.0
Deep Learning,0.0,13.0,80.0,358.0,196.0,647.0


In [ ]:
df.loc[df['skills'].str.contains('cloud', case=False, na=False), 'content'].value_counts()